In [3]:
import os
import h5py
import numpy as np
import pandas as pd
import time 

In [2]:
os.mkdir('/home/jan/Documents/Voxel_Erstellung/HDFs/New')

In [16]:
def dock_df_to_zero(df, minX, minY):
    if minX >= 0 and minY >=0:
        df['x'] = df['x'] - minX
        df['y'] = df['y'] - minY
    elif minX < 0 and minY <0:
        df['x'] = df['x'] + abs(minX)
        df['y'] = df['y'] + abs(minY)
    elif minX >= 0 and minY <0:
        df['x'] = df['x'] - minX
        df['y'] = df['y'] + abs(minY)
    elif minX < 0 and min >= 0:
        df['x'] = df['x'] + abs(minX)
        df['y'] = df['y'] - minY
    return df
#-----------------------------------------------------------------------
def get_true_min_maxX (h5_path, part_name, max_slice_number):

    minX = []
    maxX = []
    for num_slice in range(max_slice_number):
        with h5py.File(h5_path,'r') as h5:
            X_Axis = h5[part_name]['Slice'+str("{:05d}".format(num_slice+1))]['X-Axis']
            x_axis_array = np.array(X_Axis)
            minX.append(x_axis_array.min())
            maxX.append(x_axis_array.max())
    minX_array = np.asarray(minX)
    maxX_array = np.asarray(maxX)
    return minX_array.min(), maxX_array.max()
#-----------------------------------------------------------------------
def get_true_min_maxY (h5_path, part_name, max_slice_number):

    minY = []
    maxY = []
    for num_slice in range(max_slice_number):
        with h5py.File(h5_path,'r') as h5:
            Y_Axis = h5[part_name]['Slice'+str("{:05d}".format(num_slice+1))]['Y-Axis']
            y_axis_array = np.array(Y_Axis)
            minY.append(y_axis_array.min())
            maxY.append(y_axis_array.max())
    minY_array = np.asarray(minY)
    maxY_array = np.asarray(maxY)
    return minY_array.min(), maxY_array.max()
#-----------------------------------------------------------------------
def get_2D_data_from_h5_filtered(h5_path, part_name, Slice_name, mode):
    #Step 1: getting the data from the h5
    start_time = time.time()
    with h5py.File(h5_path,'r') as h5:
        X_Axis = h5[part_name][Slice_name]['X-Axis']
        Y_Axis = h5[part_name][Slice_name]['Y-Axis']
        Area = h5[part_name][Slice_name]['Area']
        Intensity = h5[part_name][Slice_name]['Intensity']

        X_Axis_size = X_Axis.size
        Y_Axis_size = Y_Axis.size
        Area_size = Area.size
        Intensity_size = Intensity.size

        #if dimensions aren't equal the following code block is entered
        if not X_Axis_size == Y_Axis_size == Area_size == Intensity_size:

            #determine the lowest value among the different sizes
            size_arr = np.array([X_Axis_size, Y_Axis_size, Area_size, Intensity_size])
            min_size = size_arr.min()

            if X_Axis_size != min_size:
                diff_size_x = X_Axis_size - min_size #calculating the difference between the actual value and the minimum and substracting it from the array
                X_Axis_new = np.delete(X_Axis, -diff_size_x)
                X_Axis = X_Axis_new
                X_Axis_size = X_Axis.size

            if Y_Axis_size != min_size:
                diff_size_y = Y_Axis_size - min_size
                Y_Axis_new = np.delete(Y_Axis, -diff_size_y)
                Y_Axis = Y_Axis_new
                Y_Axis_size = Y_Axis.size

            if Area_size != min_size:
                diff_size_area = Area_size - min_size
                Area_new = np.delete(Area, -diff_size_area)
                Area = Area_new
                Area_size = Area.size

            if Intensity_size != min_size:
                diff_size_intensity = Intensity_size - min_size
                Intensity_new = np.delete(Intensity, -diff_size_intensity)
                Intensity = Intensity_new
                Intensity_size = Intensity.size


        #by reducing all the dimensions to the minimum equal dimensions are guaranteed
        #there is a risk of deleting more than just one datapoint without noticing -> maybe add an alert after more than 5(?) while iterations
        help_arr = np.column_stack((X_Axis, Y_Axis, Area, Intensity))
        df_raw = pd.DataFrame(help_arr, columns=['x','y','area','intensity'])

    #Step 2: change floats to ints and remove duplicates
    df_int = df_raw.astype(int).drop_duplicates()

    #remove all rows with 0 for area and intensity
    df_int = df_int.loc[(df_int['area'] != 0) & (df_int['intensity'] != 0)]


    #Step 3: Get a df with all the rows where a certain x,y combination occurs multiple times
    df_multi_xy = df_int[df_int.duplicated(['x','y'], keep = False)].reset_index()

    #Step 4: get a new df out of df_multi_xy with x,y and mean/max of area and intensity for all x,y occurences
    df_compact = pd.DataFrame(columns=['x','y','area','intensity']) #initialize df_compact

    print("till iterating from {} {} seconds ---".format (Slice_name,time.time() - start_time))
    for ind in range (df_multi_xy.shape[0]):
        if mode == 'mean':
            area_mean = df_multi_xy.loc[(df_multi_xy['x']== df_multi_xy.iloc[ind]['x']) & (df_multi_xy['y'] == df_multi_xy.iloc[ind]['y'])]['area'].mean().astype(int)
            intensity_mean = df_multi_xy.loc[(df_multi_xy['x']== df_multi_xy.iloc[ind]['x']) & (df_multi_xy['y'] == df_multi_xy.iloc[ind]['y'])]['intensity'].mean().astype(int)
            df_compact = df_compact.append({'x': df_multi_xy.iloc[ind]['x'], 'y':df_multi_xy.iloc[ind]['y'], 'area':area_mean , 'intensity':intensity_mean}, ignore_index=True)
        if mode == 'max':
            area_max = df_multi_xy.loc[(df_multi_xy['x']== df_multi_xy.iloc[ind]['x']) & (df_multi_xy['y'] == df_multi_xy.iloc[ind]['y'])]['area'].max().astype(int)
            intensity_max = df_multi_xy.loc[(df_multi_xy['x']== df_multi_xy.iloc[ind]['x']) & (df_multi_xy['y'] == df_multi_xy.iloc[ind]['y'])]['intensity'].max().astype(int)
            df_compact = df_compact.append({'x': df_multi_xy.iloc[ind]['x'], 'y':df_multi_xy.iloc[ind]['y'], 'area':area_max , 'intensity':intensity_max}, ignore_index=True)
    df_compact = df_compact.drop_duplicates()

    #Step 5: remove df_multi_xy from df_int and append df_compact
    df_multi_xy_removed = df_int.drop(df_int[df_int.duplicated(['x','y'], keep = False)].index)

    df_final = df_multi_xy_removed.append(df_compact)
    print("df creation for {} took {} seconds ---".format (Slice_name,time.time() - start_time))
    return (df_final)
#-----------------------------------------------------------------------------------------------------

def create_single_voxel_df (current_n_vox_x, current_n_vox_y, voxel_size, df):
    x_min_voxel = current_n_vox_x * voxel_size
    x_max_voxel = (current_n_vox_x + 1)*voxel_size
    y_min_voxel = current_n_vox_y * voxel_size
    y_max_voxel = (current_n_vox_y + 1)*voxel_size

    x_axis_voxel_df =  np.repeat(np.arange(x_min_voxel,x_max_voxel,1),voxel_size)
    y_axis_voxel_df =  np.tile(np.arange(y_min_voxel,y_max_voxel,1),voxel_size)
    Zero_array = np.zeros(voxel_size*voxel_size, dtype=int)

    help_arr = np.column_stack((x_axis_voxel_df, y_axis_voxel_df, Zero_array, Zero_array))
    df_voxel = pd.DataFrame(help_arr, columns=['x','y','area','intensity'])


    if df[(df['x'] > x_min_voxel ) & (df['x'] < x_max_voxel ) & (df['y'] > y_min_voxel) & (df['y'] < y_max_voxel)].shape[0] != 0:
        df_voxel_added = df_voxel.append(df[(df['x'] > x_min_voxel ) & (df['x'] < x_max_voxel ) & (df['y'] > y_min_voxel) & (df['y'] < y_max_voxel)])
        df_voxel_wo_dupl = df_voxel_added.drop_duplicates(['x','y'], keep = 'last')
        df_voxel_final = df_voxel_wo_dupl.sort_values(by=['x','y'])

    else:
        df_voxel_final = df_voxel

    return df_voxel_final

In [9]:
num_layers_per_voxel = 10
num_voxels_x = 20
num_voxels_y = 20
path_buildjob_h5 = '/home/jan/Documents/CodeTDMStoHDF/Ausgangsdaten/examplerRun.h5'
path_voxel_h5_folder = '/home/jan/Documents/Voxel_Erstellung/HDFs/Versuchslauf_1'
#name_voxel_h5_file = 'asdasd_2.hdf5'
part_name = '0_00003_Canti3_cls'
mode_df = 'mean' #way how to deal with data points which occur multiple times
voxel_size = 100
max_slice_number_part = 142 # doesn't need to be manually added
os.mkdir(path_voxel_h5_folder)
minX = int(get_true_min_maxX(path_buildjob_h5, part_name, max_slice_number_part)[0])
maxX = int(get_true_min_maxX(path_buildjob_h5, part_name, max_slice_number_part)[1])
minY = int(get_true_min_maxY(path_buildjob_h5, part_name, max_slice_number_part)[0])
maxY = int(get_true_min_maxY(path_buildjob_h5, part_name, max_slice_number_part)[1])

In [14]:
def create_single_vox_layer (num_z):
    path = path_voxel_h5_folder + '/Voxel_layer:_{}.hdf5'.format(num_z)
    start_time_1 = time.time()
    for num_slice in range(num_layers_per_voxel*num_z, num_layers_per_voxel*(num_z+1)):
        start_time_2 = time.time()
        print('num_slice: ' + str(num_slice))
        #start_time = time.time()
        # getting the data of the part_hdf5
        df_not_docked = get_2D_data_from_h5_filtered(path_buildjob_h5, part_name, 'Slice' + str("{:05d}".format(num_slice+1)), mode_df) #"{:05d}" -> 1 becomes 00001 for accessibility in h5 file
        df = dock_df_to_zero(df_not_docked, minX, minY) #docking the values of the dataframe to 0

        for n_vox_y_init in range(num_voxels_y): #iterating over number of voxels in y-direction
            #print('n_vox_y_init: ' + str(n_vox_y_init))
            for n_vox_x_init in range(num_voxels_x):#iterating over number of voxels in x-direction
                #print('n_vox_x_init: '+ str(n_vox_x_init))
                df_voxel_final = create_single_voxel_df(n_vox_x_init, n_vox_y_init, voxel_size, df)

                
                #check if File is already existing -> path still needs to be defined 
                if not os.path.isfile(path):
                    voxel_hdf = h5py.File(path, "w")
                    voxel_hdf.close()
                    
                with h5py.File(path, "a") as voxel_hdf:
                    #creating a voxel with the numbers of voxels in both direction in its name and filling it with data
                    #if group is already existing don't create a new group
                    if 'voxel_{}_{}_{}'.format(n_vox_x_init,n_vox_y_init, num_z) not in voxel_hdf:
                        voxel_hdf.create_group('voxel_{}_{}_{}'.format(n_vox_x_init,n_vox_y_init,num_z))
                    voxel_hdf['voxel_{}_{}_{}'.format(n_vox_x_init,n_vox_y_init,num_z)].create_group('slice_{}'.format(num_slice-num_z*num_layers_per_voxel)) #-num_z*num_slices_vox wegen
                    voxel_hdf['voxel_{}_{}_{}'.format(n_vox_x_init,n_vox_y_init,num_z)]['slice_{}'.format(num_slice-num_z*num_layers_per_voxel)].create_dataset('X-Axis',data = np.repeat(np.arange(0,voxel_size,1),voxel_size))
                    voxel_hdf['voxel_{}_{}_{}'.format(n_vox_x_init,n_vox_y_init,num_z)]['slice_{}'.format(num_slice-num_z*num_layers_per_voxel)].create_dataset('Y-Axis',data = np.tile(np.arange(0,voxel_size,1),voxel_size))
                    voxel_hdf['voxel_{}_{}_{}'.format(n_vox_x_init,n_vox_y_init,num_z)]['slice_{}'.format(num_slice-num_z*num_layers_per_voxel)].create_dataset('Area', data = df_voxel_final['area'].values.astype(int))
                    voxel_hdf['voxel_{}_{}_{}'.format(n_vox_x_init,n_vox_y_init,num_z)]['slice_{}'.format(num_slice-num_z*num_layers_per_voxel)].create_dataset('Intensity', data = df_voxel_final['intensity'].values.astype(int))
        print('filling slice {} took {} s'.format(num_slice, time.time() - start_time_2))
    print("layer filling took %s seconds ---" % (time.time() - start_time_1))

In [17]:
create_single_vox_layer(1)

num_slice: 10
till iterating from Slice00011 0.05177474021911621 seconds ---
df creation for Slice00011 took 78.39901161193848 seconds ---
filling slice 10 took 84.39902448654175 s
num_slice: 11
till iterating from Slice00012 0.04531288146972656 seconds ---
df creation for Slice00012 took 71.6744430065155 seconds ---
filling slice 11 took 77.27283692359924 s
num_slice: 12
till iterating from Slice00013 0.04584527015686035 seconds ---
df creation for Slice00013 took 71.09422206878662 seconds ---
filling slice 12 took 76.70865082740784 s
num_slice: 13
till iterating from Slice00014 0.03506803512573242 seconds ---
df creation for Slice00014 took 75.26889061927795 seconds ---
filling slice 13 took 80.92940354347229 s
num_slice: 14
till iterating from Slice00015 0.03848981857299805 seconds ---
df creation for Slice00015 took 75.85409879684448 seconds ---
filling slice 14 took 81.47097849845886 s
num_slice: 15
till iterating from Slice00016 0.0344080924987793 seconds ---
df creation for Slic